从两个渠道下载上市公司年报

- 优点==》数据全，所有历史的年报都有
- 缺点==》下载速度慢，PDF格式，导致有些下载文件不能提取出文本来


# 深交所和上交所股票代码的区别
>股票代码用数字表示股票的不同含义。股票代码除了区分各种股票，也有其潜在的意义。沪市股票买卖的代码是以600或601打头，如：运盛实业：股票代码是600股票代码767。中国国航：股票代码是601111。B股买卖的代码是以900打头，如：上电B股：代码是900901。深市A股票买卖的代码是以000打头，如：顺鑫农业：股票代码是000860。B股买卖的代码是以200打头，如：深中冠B股，代码是200018。

>沪市新股申购的代码是以730打头。如：中信证券：申购的代码是730030。深市新股申购的代码与深市股票买卖代码一样，如：中信证券在深市市值配售代码是003030。配股代码，沪市以700打头，深市以080打头。如：运盛实业配股代码是700767。深市草原兴发配股代码是080780。中小板股票代码以002打头，如：东华合创股票代码是002065。创业板股票代码以300打头，如：探路者股票代码是：300005


# 深圳证券交易所：
http://www.szse.cn/disclosure/listed/notice/index.html


# 上海深圳交易所
http://www.sse.com.cn/disclosure/listedinfo/announcement/





In [2]:
#!jupyter nbconvert --to script web_spider_sse_szse_pdf_urls
import requests
import re
def get_url_by_code_szse(code):
    download_url_prefix = 'http://disc.static.szse.cn/download'
    
    url = 'http://www.szse.cn/api/disc/announcement/annList'
    payload = {"seDate":["",""],"stock":["000001"],"channelCode":["listedNotice_disc"],"bigCategoryId":["010301"],"pageSize":30,"pageNum":1}
    headers = {'content-type': 'application/json',
              'Host': 'www.szse.cn',
    'Origin': 'http://www.szse.cn',
    'Referer': 'http://www.szse.cn/disclosure/listed/notice/index.html',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'      
              }
    payload['stock']=[code]
    payload['pageNum']=1
    r = requests.post(url, json=payload)
    ret = r.json()
    count = ret['announceCount']
    data_list = ret['data']
    # 由于限制是30每页，总数超过30，要第二次请求
    if(count>30):
        payload['pageNum']=2
        r = requests.post(url, json=payload)
        ret = r.json()
        data_list.extend(ret['data'])
        
    year_report_map={}
    for d in data_list:
        fileName = d['title']
        if(fileName.endswith('摘要')):
            continue
        if(not(fileName.endswith('年报') or fileName.endswith('年度报告'))):
            continue
        year = re.findall('\d{4}',fileName)[0]
        year_report_map[code+'_'+year]=download_url_prefix+d['attachPath']
#         print(d['secName'][0],d['title'],d['attachPath'])
    return year_report_map


# get_url_by_code_szse('000066')
        

In [3]:
from datetime import datetime,timedelta
import json
# 上海证券交易所
def get_url_by_code_sse(code):
    url = 'http://query.sse.com.cn/security/stock/queryCompanyStatementNew.do'
    params={
        'jsonCallBack': 'jsonpCallback59529',
        'isPagination': 'true',
        'productId': '600795',
        'keyWord': '',
        'isNew': '1',
        'reportType2': 'DQBG',
        'reportType': 'YEARLY',
        'beginDate': '2002-04-13',
        'endDate': '2019-04-13',
        'pageHelp.pageSize': 25,
        'pageHelp.pageCount': 50,
        'pageHelp.pageNo': 1,
        'pageHelp.beginPage': 1,
        'pageHelp.cacheSize': 1,
        'pageHelp.endPage': 5,
        '_': 1555167217524
    }
    headers = {'Referer': 'http://www.sse.com.cn/disclosure/listedinfo/announcement/',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
    data_list = []
    params['productId']=code
    beginDate=datetime.strptime('2003-01-01','%Y-%m-%d')
    _endDate= datetime.now()
    _3years = timedelta(days=365*3-1)
    while beginDate< _endDate:
        params['beginDate']=beginDate.strftime('%Y-%m-%d')
        beginDate += _3years
        params['endDate']=beginDate.strftime('%Y-%m-%d')
        r = requests.get(url,headers=headers,params=params)
        json_str = r.text[r.text.index('(')+1:-1]
        json_obj = json.loads(json_str)
        data_list.extend(json_obj['result'])


    year_report_map={}
    for d in data_list:
        fileName = d['title']
        if(fileName.endswith('摘要')):
            continue
        if(not(fileName.endswith('年报') or fileName.endswith('年度报告'))):
            continue
#         print(d['bulletin_Year'],d['title'],d['URL'])
        year_report_map[code+'_'+d['bulletin_Year']]='http://www.sse.com.cn'+d['URL']
        
    return year_report_map


#get_url_by_code_sse('600795')

In [53]:
#url = 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2010-03-31/600795_2009_n.pdf'
url = 'http://disc.static.szse.cn/download/disc/disk01/finalpage/2008-04-19/cda5e777-cc3c-4934-9971-99ce9e4f6fe0.PDF'
def down_pdf(code_year,url,pdf_dir='./',txt_dir='./'):
    filename = pdf_dir+code_year+'.pdf'
    try:
        r = requests.get(url, stream=True)
        with open(filename, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=102400):
                fd.write(chunk)
    except:
        pass
    
    return filename



In [4]:

import os
import pandas as pd
import re
from tqdm import tqdm

df1 = pd.read_excel('研究数据0410.xlsx',index_col=0,
            dtype={'Symbol': str})

df1.head()


,Symbol,EndDate,Stknme,Listexg,Nindcd,Area,Listdt,Ucpd,Ucsvr,Nshld,...,F050203B,F050204C,F100501A,F100801A,F100802A,F100901A,F100902A,F100903A,F100904A,备注
0,没有单位',没有单位',没有单位',没有单位',没有单位',没有单位',没有单位',没有单位',没有单位',人',...,没有单位',没有单位',没有单位',没有单位',没有单位',没有单位',没有单位',没有单位',没有单位',NaN
1,证券代码',统计截止日期',证券简称',上市交易所',行业代码',地区',上市日期',实际控制人是否担任董事长或总经理',实际控制人拥有上市公司控制权比例(%)',股东总数',...,总资产净利润率（ROA）C',总资产净利润率（ROA）TTM',市值有形资产比',市值A',市值B',托宾Q值A',托宾Q值B',托宾Q值C',托宾Q值D',只取12月年终的数据
2,000001,2017-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.007478,0.007478,0.070564,3.25469e+12,3.25479e+12,0.0703,0.070566,1.00194,1.00574,NaN
3,000001,2016-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.008277,0.008277,0.053125,2.9143e+12,2.90751e+12,0.052905,0.053127,0.984452,0.988582,NaN
4,000001,2015-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.009317,0.009317,0.068774,2.51545e+12,2.51721e+12,0.068429,0.068772,1.00401,1.00906,NaN


In [5]:
df2 = df1[['Symbol','EndDate']] 
data_list = df2.to_dict('records')
code_year_map={}
for d in data_list:
    code = d['Symbol']
    if not re.match('\d{6}',code):
        continue
    if code not in code_year_map:
        code_year_map[code]=[]
    code_year_map[code].append(d['EndDate'][:4])


In [ ]:
"""
下载文件的url
以格式 保存
code 年份 url格式保存
"""
import time
import json
keys = list(code_year_map.keys())
with open('pdf_urls.json') as json_file:  
    pdf_urls = json.load(json_file)

counter=0
for code in tqdm(keys):
    if code+'_2017' in pdf_urls:
        print(f'{code} already download urls,jump.....')
        continue
    counter+=1
    times=0
    while times<=3:
        times += 1
        try:
            #获取文件的url
            if code[0]=='6':
                year_report_map=get_url_by_code_sse(code)
            else:
                year_report_map=get_url_by_code_szse(code)
        except Exception as e:
            print('get year_report_map error:',e)
            time.sleep(1)
            continue
        for k in year_report_map.keys():
            pdf_urls[k] = year_report_map[k]
    if counter % 20 == 0:
        print('*'*40,'write json file')
        with open('pdf_urls.json','w') as json_file:  
            json.dump(pdf_urls,json_file, sort_keys=True, indent=2)
    
print('*'*40,'write json file')
with open('pdf_urls.json','w') as json_file:  
    json.dump(pdf_urls,json_file, sort_keys=True, indent=2)
        


  1%|          | 20/2823 [02:00<4:42:22,  6.04s/it]

**************************************** write json file



  1%|▏         | 40/2823 [04:00<4:27:19,  5.76s/it]

**************************************** write json file



  2%|▏         | 60/2823 [06:01<4:46:29,  6.22s/it]

...


 13%|█▎        | 360/2823 [35:55<3:56:38,  5.76s/it]

**************************************** write json file



 13%|█▎        | 380/2823 [37:57<4:13:59,  6.24s/it]

**************************************** write json file



 14%|█▍        | 390/2823 [38:56<3:53:43,  5.76s/it]

In [ ]:
# 检查文件是否存在
# 根据号码判断是深交所还是上交所
base_dir='../scraping/content/'
pdf_dir ='../scraping/pdf/'
ok,pdf_ok,nok=0,0,0
sse,szse=0,0
import time  
from PyPDF2 import PdfFileWriter, PdfFileReader
keys = list(code_year_map.keys())

def checkPdf(pdf_file):
    """检查pdf文件的合法性"""
    if not os.path.exists(pdf_file):
        return False
    try:
        pdfFileReader = PdfFileReader(pdf_file)
        if pdfFileReader.isEncrypted:
            pdfFileReader.decrypt('')
        if pdfFileReader.getDocumentInfo()  is not None:
            return True
    except:
        pass
    return False
    
    
for code in tqdm(keys):
    try:
        if code[0]=='6':
            sse +=1
        else:
            szse +=1

        lost_file_list = []

        for year in code_year_map[code]:
            f=base_dir+code+'_'+year+'.txt'
            fsize=0
            try:
                fsize = os.path.getsize(f)
            except:
                pass

            if fsize>1024*200:
                ok+=1
                continue
            pdf_file=pdf_dir+code+'_'+year+'.pdf'
            if checkPdf(pdf_file):
                print('+'*15,f'校验文件({pdf_file})成功')
                pdf_ok +=1
                continue

            lost_file_list.append(code+'_'+year)
            nok+=1
    #         print(f,'not exists of too small',fsize)

        if len(lost_file_list)==0:
            continue
        try:
            #获取文件的url
            if code[0]=='6':
                year_report_map=get_url_by_code_sse(code)
            else:
                year_report_map=get_url_by_code_szse(code)
        except Exception as e:
            print('get year_report_map error:',e)

        for f in lost_file_list:
            if f not in year_report_map:
                print(f'{f} not in year_report_map','*'*100)
                continue 
            url = year_report_map[f]
            times = 1
            pdf_file = pdf_dir+f+'.pdf'
            if checkPdf(pdf_file):
                continue

            while times<= 3:
                t0=time.time()
                print(f'开始下载文件：{url},times={times}')
                pdf_file = down_pdf(f,url,pdf_dir)
                fsize=1
                try:
                    fsize = os.path.getsize(pdf_file)
                except:
                    pass
                print(f'下载文件({pdf_file})成功，用时(秒):{time.time()-t0},times={times},fsize={fsize/1024}KB')
                t0=time.time()
                if checkPdf(pdf_file):
                    print('+'*30,f'校验文件({pdf_file})成功')
                    times +=5
                else:
                    print('-'*30,f'校验文件({pdf_file})失败，times={times}')
    #                 if os.path.exists(pdf_file):
    #                     os.remove(pdf_file)
                    times +=1
                    time.sleep(3)
    except Exception as e:
        print('for code in tqdm(keys)',e)
    
    
print(sse,szse,len(code_year_map.keys()))       
print(ok,pdf_ok,nok,ok+nok)    
       


In [87]:
# # checkPdf(pdf_dir+'000011_2004'+'.pdf')
# # checkPdf()
# pdfFileReader = PdfFileReader('../scraping/pdf/000006_2003.pdf',strict = False)

# if pdfFileReader.isEncrypted:
#     pdfFileReader.decrypt('')
# # # 获取 PDF 文件的文档信息
# documentInfo = pdfFileReader.getDocumentInfo()
# print('documentInfo = %s' % documentInfo)
# # # 获取页面布局
# # pageLayout = pdfFileReader.getPageLayout()
# # print('pageLayout = %s ' % pageLayout)

# # # 获取页模式
# # pageMode = pdfFileReader.getPageMode()
# # print('pageMode = %s' % pageMode)

# # xmpMetadata = pdfFileReader.getXmpMetadata()
# # print('xmpMetadata  = %s ' % xmpMetadata)

# # 获取 pdf 文件页数
# pageCount = pdfFileReader.getNumPages()

# print('pageCount = %s' % pageCount)

888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888
documentInfo = {'/ModDate': "D:20040420203556+08'00'", '/CreationDate': "D:20040420110634+08'00'", '/Title': 'untitled', '/Producer': 'Acrobat Distiller 6.0.1 (Windows)'}
